In [1]:
from itertools import combinations

import os
import sys
from pathlib import Path

sys.path.append(str((Path(os.path.abspath("")) / "..").resolve()))
from sudoku_variants import Sudoku, SudokuAI, SudokuConst
from sudoku_variants.rule import Orthogonal, SubBoard, Knight, King, Consecutive, Jigsaw
from sudoku_variants.func import rules as R


def standard_rules():
    return [Orthogonal(), SubBoard()]


def variant_rules():
    return [Knight(), King(), Consecutive(), Jigsaw()]

In [2]:
import timeit


def profile(number: int):
    def inner(func):
        def wrapper(*args, **kwargs):
            def run():
                func(*args, **kwargs)

            time = timeit.timeit("run()", globals=locals(), number=number)
            print(f"Time: {time:.4f}s, {number}x | {time/number:.4f}s/time")

        return wrapper

    return inner

### AI

In [3]:
@profile(10)
def run(rules):
    board = [[0 for _ in range(SudokuConst.NUM_COL)] for _ in range(SudokuConst.NUM_ROW)]
    sudoku = Sudoku(board, rules)
    SudokuAI.solve(sudoku)


for length in range(len(variant_rules()) + 1):
    for subset in combinations(variant_rules(), length):
        print(R.to_name(list(subset)))
        rules = R.with_standard_rules(list(subset))

        # board = [[0 for _ in range(SudokuConst.NUM_COL)] for _ in range(SudokuConst.NUM_ROW)]
        # sudoku = Sudoku(board, rules)
        # R.populate_initial_data(rules)
        # SudokuAI.solve(sudoku)
        # R.extract_data_from_board(rules, sudoku.board)

        run(rules)


Time: 0.0300s, 10x | 0.0030s/time
Knight
Time: 1.9678s, 10x | 0.1968s/time
King
Time: 0.0528s, 10x | 0.0053s/time
Consecutive
Time: 0.0318s, 10x | 0.0032s/time
Jigsaw
Time: 0.0263s, 10x | 0.0026s/time
Knight, King
Time: 2.1149s, 10x | 0.2115s/time
Knight, Consecutive
Time: 1.1540s, 10x | 0.1154s/time
Knight, Jigsaw
Time: 1.3599s, 10x | 0.1360s/time
King, Consecutive
Time: 0.0718s, 10x | 0.0072s/time
King, Jigsaw
Time: 0.1735s, 10x | 0.0173s/time
Consecutive, Jigsaw
Time: 0.0378s, 10x | 0.0038s/time
Knight, King, Consecutive
Time: 2.3619s, 10x | 0.2362s/time
Knight, King, Jigsaw
Time: 2.0712s, 10x | 0.2071s/time
Knight, Consecutive, Jigsaw
Time: 1.0335s, 10x | 0.1033s/time
King, Consecutive, Jigsaw
Time: 0.0710s, 10x | 0.0071s/time
Knight, King, Consecutive, Jigsaw
Time: 1.5736s, 10x | 0.1574s/time


In [4]:
@profile(10)
def run(rules, to_erase):
    SudokuAI.generate(rules, max_erased=to_erase)


rules = [Orthogonal(), SubBoard()]
for to_erase in [30, 40, 50, 60]:
    print(to_erase)
    run(rules, to_erase)

30
Time: 0.1669s, 10x | 0.0167s/time
40
Time: 0.3420s, 10x | 0.0342s/time
50
Time: 4.4955s, 10x | 0.4496s/time
60


KeyboardInterrupt: 

In [ ]:
@profile(1)
def run():
    sudoku = SudokuAI.generate([Orthogonal(), SubBoard(), Knight(), King(), Consecutive()], max_erased=40)
    print(sudoku)


run()

Rules: Orthogonal, SubBoard, Orthogonal, SubBoard, Knight, King, Consecutive
+-------+-------+-------+
| 8 9 4 | * * * | 1 * 6 |
| * 5 * | 8 9 4 | 2 3 * |
| * 3 * | 1 * * | * 9 * |
+-------+-------+-------+
| 6 * * | 4 2 3 | 7 1 5 |
| 7 * 5 | 6 8 * | 4 * * |
| 4 * 3 | * * 5 | * 8 * |
+-------+-------+-------+
| 5 * * | 9 * * | * * * |
| 3 * * | * 6 * | * 4 2 |
| * 4 2 | 3 7 1 | * * * |
+-------+-------+-------+
Time: 0.2229s, 1x | 0.2229s/time
